In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import gc

In [2]:
ls

Data_Exploration.ipynb              application_train.csv
Data_Exploration_FullSet.ipynb      bureau.csv
DecisionTree_Submission.ipynb       bureau_balance.csv
Feature_Eng.ipynb                   credit_card_balance.csv
Feature_Eng_take2.ipynb             installments_payments.csv
HomeCredit_columns_description.csv  lgbm_sub.csv
LGBM_hyperparameterization.ipynb    previous_application.csv
LightGBM.ipynb                      sample_submission.csv
Model Performance.ipynb             test_feat_eng.csv
POS_CASH_balance.csv                train_feat_eng.csv
application_test.csv


In [3]:
# load train and test sets
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')

In [4]:
train['AMT_CREDIT_TO_ANNUITY_RATIO'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
test['AMT_CREDIT_TO_ANNUITY_RATIO'] = test['AMT_CREDIT'] / test['AMT_ANNUITY']

In [5]:
# get number of past loans per customer 
bureau = pd.read_csv('bureau.csv')
B = bureau
grp = B[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'NUM_PREV_LOANS'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')

In [6]:
# get percentage of active loans: more loans can potential mean a default

B['CREDIT_ACTIVE_BINARY'] = B['CREDIT_ACTIVE']
B['CREDIT_ACTIVE_BINARY'] = B.apply(lambda x: x.CREDIT_ACTIVE == 'Closed', axis = 1)

# Calculate mean number of loans that are ACTIVE per CUSTOMER 
grp = B.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del B['CREDIT_ACTIVE_BINARY']
gc.collect()

0

In [7]:
B['AMT_CREDIT_SUM_DEBT'] = B['AMT_CREDIT_SUM_DEBT'].fillna(0)
B['AMT_CREDIT_SUM'] = B['AMT_CREDIT_SUM'].fillna(0)

grp1 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

B = B.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
B = B.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
del grp1, grp2
gc.collect()

B['DEBT_CREDIT_RATIO'] = B['TOTAL_CUSTOMER_DEBT']/B['TOTAL_CUSTOMER_CREDIT']

del B['TOTAL_CUSTOMER_DEBT'], B['TOTAL_CUSTOMER_CREDIT']
gc.collect()

0

In [8]:
# merge new columns
B = B[["SK_ID_CURR","NUM_PREV_LOANS", "ACTIVE_LOANS_PERCENTAGE", "DEBT_CREDIT_RATIO"]]
B = B.drop_duplicates()
train = train.merge(B, on = 'SK_ID_CURR', how = 'left')
test = test.merge(B, on = 'SK_ID_CURR', how = 'left')

In [10]:
# # get debt/credit ratio
# credit_card_balance = pd.read_csv('credit_card_balance.csv')
# credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [11]:
# # lets work with some previous applicaton data
# prev_applications = pd.read_csv('previous_application.csv')
# prev_applications.head()

In [12]:
# # POS data: interesting data is their monthly balance, number of installments left
# pos_balance = pd.read_csv('POS_CASH_balance.csv')
# pos_balance

In [13]:
train['SK_ID_CURR'].duplicated().any()

False

In [14]:
# output engineered CSVs 
train.to_csv("train_feat_eng.csv")
test.to_csv("test_feat_eng.csv")